# BCB Final Project - a.y. 17/18

### Fiaschi Lorenzo
### Franco Danilo

<ol> <b>TODO</b>
    <li>problem presentation (dataset, objectives)</li>
    <li>mkl procedure overview</li>
    <li>chosen model description (alignment model, cortes approach)</li>
    <li>algorithm pipeline (with a major point in the cross-validation decisions, how it is shuffled...)</li>
    <li>approach settings</li>
    <li>performances over toy (with description on how it is generated</li>
    <li>integration needed? comparison with single dataset model</li>
    <li>performances over real dataset</li>
    <li>what we can learn from the best configuration (sparsity)</li>
    <li>kernel integration on the only meaningful dataset (clinical)</li>
    <li>overview on the regression approach</li>
    <li>regression performances with data integration</li>
    <li>single kernel approach after regression results</li>
    <li>final comments over the results</li>
</ol>